In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

import statsmodels.api as sm
from prophet import Prophet

import xgboost as xgb
from darts import TimeSeries
from darts.models import ExponentialSmoothing, ARIMA, RNNModel

sns.set(style = "whitegrid")
plt.style.use("fivethirtyeight")

c:\Users\Lenovo\Documents\Projects\PMSF\PMSF\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
import pandas as pd
train_file_path = r'C:\Users\Lenovo\Documents\Projects\PMSF\dataset\rossmann-store-sales\train.csv'
test_file_path = r'C:\Users\Lenovo\Documents\Projects\PMSF\dataset\rossmann-store-sales\test.csv'
store_file_path = r'C:\Users\Lenovo\Documents\Projects\PMSF\dataset\rossmann-store-sales\store.csv'

# Load the datasets
train_df = pd.read_csv(train_file_path)
test_df = pd.read_csv(test_file_path)
store_df = pd.read_csv(store_file_path)

# Ensure the data is loaded correctly
print("Train DataFrame Head:")
print(train_df.head())
print("\nTest DataFrame Head:")
print(test_df.head())
print("\nStore DataFrame Head:")
print(store_df.head())


Train DataFrame Head:
   Store  DayOfWeek        Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5  2015-07-31   5263        555     1      1            0   
1      2          5  2015-07-31   6064        625     1      1            0   
2      3          5  2015-07-31   8314        821     1      1            0   
3      4          5  2015-07-31  13995       1498     1      1            0   
4      5          5  2015-07-31   4822        559     1      1            0   

   SchoolHoliday  
0              1  
1              1  
2              1  
3              1  
4              1  

Test DataFrame Head:
   Id  Store  DayOfWeek        Date  Open  Promo StateHoliday  SchoolHoliday
0   1      1          4  2015-09-17   1.0      1            0              0
1   2      3          4  2015-09-17   1.0      1            0              0
2   3      7          4  2015-09-17   1.0      1            0              0
3   4      8          4  2015-09-17   1.0      1           

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4572\821647093.py:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_file_path)


In [3]:
# Merge train and store data
train_df = train_df.merge(store_df, on='Store', how='left')
test_df = test_df.merge(store_df, on='Store', how='left')

In [4]:
print("\nMerged Train DataFrame Head:")
print(train_df.head())
print("\nMerged Test DataFrame Head:")
print(test_df.head())


Merged Train DataFrame Head:
   Store  DayOfWeek        Date  Sales  Customers  Open  Promo StateHoliday  \
0      1          5  2015-07-31   5263        555     1      1            0   
1      2          5  2015-07-31   6064        625     1      1            0   
2      3          5  2015-07-31   8314        821     1      1            0   
3      4          5  2015-07-31  13995       1498     1      1            0   
4      5          5  2015-07-31   4822        559     1      1            0   

   SchoolHoliday StoreType Assortment  CompetitionDistance  \
0              1         c          a               1270.0   
1              1         a          a                570.0   
2              1         a          a              14130.0   
3              1         c          c                620.0   
4              1         a          a              29910.0   

   CompetitionOpenSinceMonth  CompetitionOpenSinceYear  Promo2  \
0                        9.0                    2008.0  

In [5]:
#handling missing values
print(train_df.isnull().sum())
print(test_df.isnull().sum())

#Filling missing values in competitionDistance with a large number 
train_df['CompetitionDistance'].fillna(train_df['CompetitionDistance'].max(), inplace=True)
test_df['CompetitionDistance'].fillna(test_df['CompetitionDistance'].max(), inplace=True)

#Fill missing values in Promo2SinceYear and Promo2SinceWeek with 0
train_df['Promo2SinceYear'].fillna(0, inplace=True)
train_df['Promo2SinceWeek'].fillna(0, inplace=True)
test_df['Promo2SinceYear'].fillna(0, inplace=True)
test_df['Promo2SinceWeek'].fillna(0, inplace=True)

#Fill missing values in PromoInterval with 'None'
train_df['PromoInterval'].fillna('None', inplace=True)
test_df['PromoInterval'].fillna('None', inplace=True)

Store                             0
DayOfWeek                         0
Date                              0
Sales                             0
Customers                         0
Open                              0
Promo                             0
StateHoliday                      0
SchoolHoliday                     0
StoreType                         0
Assortment                        0
CompetitionDistance            2642
CompetitionOpenSinceMonth    323348
CompetitionOpenSinceYear     323348
Promo2                            0
Promo2SinceWeek              508031
Promo2SinceYear              508031
PromoInterval                508031
dtype: int64
Id                               0
Store                            0
DayOfWeek                        0
Date                             0
Open                            11
Promo                            0
StateHoliday                     0
SchoolHoliday                    0
StoreType                        0
Assortment              

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4572\2212314993.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df['CompetitionDistance'].fillna(train_df['CompetitionDistance'].max(), inplace=True)
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4572\2212314993.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are settin

In [6]:
#Feature Engineering
train_df['Date'] = pd.to_datetime(train_df['Date'])
test_df["Date"] = pd.to_datetime(test_df['Date'])

#Extract year, month, day and week of year from Date
train_df['Year'] = train_df['Date'].dt.year
train_df['Month'] = train_df['Date'].dt.month
train_df['Day'] = train_df["Date"].dt.day
train_df['WeekOfYear']= train_df['Date'].dt.isocalendar().week

test_df['Year'] = test_df['Date'].dt.year
test_df['Month'] = test_df['Date'].dt.month
test_df['Day'] = test_df['Date'].dt.day
test_df['WeekOfYear'] = test_df['Date'].dt.isocalendar().week

#create a feature for the duration of the competition
train_df['CompetitionOpenSince']= 12* (train_df['Year'] - train_df['CompetitionOpenSinceYear']) + (train_df['Month'] - train_df['CompetitionOpenSinceMonth'])
test_df['CompetitionOpenSince'] = 12 * (test_df['Year'] - test_df['CompetitionOpenSinceYear']) + (test_df['Month'] - test_df['CompetitionOpenSinceMonth'])

#Create ferature for the duration of the Promotion
train_df['Promo2Since'] = 12 * (train_df['Year'] - train_df['Promo2SinceYear']) + (train_df['WeekOfYear'] - train_df['Promo2SinceWeek'])
test_df['Promo2Since'] = 12 * (test_df['Year'] - test_df['Promo2SinceYear']) + (test_df['WeekOfYear'] - test_df['Promo2SinceWeek'])

#Replace negative values with 0
train_df["CompetitionOpenSince"] = train_df['CompetitionOpenSince'].apply(lambda x: x if x > 0 else 0)
test_df['CompetitionOpenSince'] = test_df['CompetitionOpenSince'].apply(lambda x: x if x > 0 else 0)

In [7]:
# One hot encode categorical values
train_df = pd.get_dummies(train_df, columns=['StoreType', 'Assortment', 'StateHoliday', 'PromoInterval'], prefix=['StoreType', 'Assortment', 'StateHoliday', 'PromoInterval'])
test_df = pd.get_dummies(test_df, columns=['StoreType', 'Assortment', 'StateHoliday', 'PromoInterval'], prefix=['StoreType', 'Assortment', 'StateHoliday', 'PromoInterval'])


#Remove duplicate columns if any
train_df = train_df.loc[:, ~train_df.columns.duplicated()]
test_df = test_df.loc[:, ~test_df.columns.duplicated()]

# Ensuring both train and test have the same columns after one-hot encoding
train_columns = set(train_df.columns)
test_columns = set(test_df.columns)
for col in train_columns - test_columns:
    test_df[col] = 0
for col in test_columns - train_columns:
    train_df[col] = 0
test_df = test_df[train_df.columns]


In [8]:
# Drop unnecessary columns
columns_to_drop = ['Date', 'Customers', 'CompetitionOpenSinceYear', 'CompetitionOpenSinceMonth', 'Promo2SinceYear', 'Promo2SinceWeek']
train_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')
test_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# Verify the columns have been dropped
print("\nTrain DataFrame Columns After Dropping:")
print(train_df.columns)
print("\nTest DataFrame Columns After Dropping:")
print(test_df.columns)



Train DataFrame Columns After Dropping:
Index(['Store', 'DayOfWeek', 'Sales', 'Open', 'Promo', 'SchoolHoliday',
       'CompetitionDistance', 'Promo2', 'Year', 'Month', 'Day', 'WeekOfYear',
       'CompetitionOpenSince', 'Promo2Since', 'StoreType_a', 'StoreType_b',
       'StoreType_c', 'StoreType_d', 'Assortment_a', 'Assortment_b',
       'Assortment_c', 'StateHoliday_0', 'StateHoliday_a', 'StateHoliday_b',
       'StateHoliday_c', 'PromoInterval_Feb,May,Aug,Nov',
       'PromoInterval_Jan,Apr,Jul,Oct', 'PromoInterval_Mar,Jun,Sept,Dec',
       'PromoInterval_None', 'Id'],
      dtype='object')

Test DataFrame Columns After Dropping:
Index(['Store', 'DayOfWeek', 'Sales', 'Open', 'Promo', 'SchoolHoliday',
       'CompetitionDistance', 'Promo2', 'Year', 'Month', 'Day', 'WeekOfYear',
       'CompetitionOpenSince', 'Promo2Since', 'StoreType_a', 'StoreType_b',
       'StoreType_c', 'StoreType_d', 'Assortment_a', 'Assortment_b',
       'Assortment_c', 'StateHoliday_0', 'StateHoliday_a', 'St

In [9]:
# Define features and target
X_train = train_df.drop(columns=['Sales'])
y_train = train_df['Sales']

# For test data, ensure the same columns are present
X_test = test_df[X_train.columns]


In [10]:
bool_cols = X_train.select_dtypes(include=['bool']).columns
X_train[bool_cols] = X_train[bool_cols].astype(int)
X_test[bool_cols] = X_test[bool_cols].astype(int)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4572\2450137976.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[bool_cols] = X_test[bool_cols].astype(int)


In [11]:
numerical_cols = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns


In [12]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', SimpleImputer(strategy='mean'), numerical_cols),
        ('cat', SimpleImputer(strategy='most_frequent'), categorical_cols)
    ],
    remainder='passthrough'
)

In [13]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('scaler', StandardScaler())
])

In [14]:
X_train_preprocessed = pipeline.fit_transform(X_train)
X_test_preprocessed = pipeline.transform(X_test)

In [ ]:
from sklearn.preprocessing import StandardScaler

#initialize the scaler 
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train_imputed)

# Transform the test data
X_test_scaled = scaler.fit_transform(X_test_imputed)

In [15]:
# Linear Regression
lr_model = LinearRegression()
lr_model.fit(X_train_preprocessed, y_train)
y_pred_lr = lr_model.predict(X_test_preprocessed)
mse_lr = mean_squared_error(y_train, lr_model.predict(X_train_preprocessed))
rmse_lr = np.sqrt(mse_lr)
print(f"Linear Regression RMSE: {rmse_lr}")


Linear Regression RMSE: 2525.7662538024942


In [17]:
#Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_preprocessed, y_train)
y_pred_rf = rf_model.predict(X_test_preprocessed)
mse_rf = mean_squared_error(y_train, rf_model.predict(X_train_preprocessed))
rmse_rf = np.sqrt(mse_rf)
print(f"Random Forest RMSE: {rmse_rf}")

Random Forest RMSE: 312.7414964797352


In [ ]:
#Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_preprocessed, y_train)
y_pred_rf = rf_model.predict(X_test_preprocessed)
mse_rf = mean_squared_error(y_train, rf_model.predict(X_train_preprocessed))
rmse_rf = np.sqrt(mse_rf)
print(f"Random Forest RMSE: {rmse_rf}")

Random Forest RMSE: 312.7414964797352


In [18]:
#XGBoost Regressor
xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
xgb_model.fit(X_train_preprocessed, y_train)
y_pred_xgb = xgb_model.predict(X_test_preprocessed)
mse_xgb = mean_squared_error(y_train, xgb_model.predict(X_train_preprocessed))
rmse_xgb = np.sqrt(mse_xgb)
print(f"XGBoost RMSE: {rmse_xgb}")

XGBoost RMSE: 1179.8632922800082
